# Environment Check

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan 24 21:50:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Swim V1 Transformer

In [6]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 8.7 MB/s 


In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers


import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Attention Block

In [14]:
class Attention(layers.Layer):
    def __init__(self, numpatchesdim=7, querykeydim=32):
        super(Attention, self).__init__()
        self.M2 = numpatchesdim*numpatchesdim
        self.d = querykeydim
        self.K = self.add_weight(
            shape=(self.M2, self.d), initializer="random_normal", trainable=True
        )
        self.V = self.add_weight(
            shape=(self.M2, self.d), initializer="random_normal", trainable=True
        )
        self.B = self.add_weight(
            shape=(self.M2, self.M2), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        Q = inputs
        attn = Q @ tf.transpose(self.K) / tf.math.sqrt(tf.cast(self.d, tf.float32))
        attn = tf.nn.softmax(attn) @ self.V
        return attn


### Attention Block Testing

In [15]:
numpatchesdim = 7
querykeydim = 32
attnlayer = Attention(numpatchesdim=numpatchesdim,
                      querykeydim=querykeydim)
Q = tf.ones(shape=(numpatchesdim*numpatchesdim, querykeydim))
out = attnlayer(Q)

In [16]:
out

<tf.Tensor: shape=(49, 32), dtype=float32, numpy=
array([[-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715],
       [-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715],
       [-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715],
       ...,
       [-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715],
       [-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715],
       [-0.00201054,  0.00519757, -0.00151976, ..., -0.00219627,
        -0.00044917,  0.00808715]], dtype=float32)>